## 📌 Proje Amacı

Bu çalışmada, Türkiye gayrimenkul sektörünü etkileyen temel ekonomik göstergelerin **2015–2025** dönemindeki değişimi analiz edilmek istenmektedir. Kullanılacak verilerin bir bölümü **aylık zaman serisi**, bir bölümü ise **günlük frekansta** olduğundan; projede kapsamlı bir **temizleme, dönüştürme ve birleştirme** süreci uygulanacaktır. Veri kaynaklarının genel kalitesi yeterli olmakla birlikte, ihtiyaç duyulan ek göstergeler **World Bank** ve **IMF** gibi uluslararası kurumlardan da CSV formatında temin edilebilmektedir.

Projenin ana veri kaynağı, **Türkiye Cumhuriyet Merkez Bankası (TCMB) EVDS API** sistemidir. EVDS üzerinden çekilen konut, kredi, fiyat, üretim ve diğer makroekonomik göstergeler, sektörü bütüncül bir çerçevede inceleme imkanı sunmaktadır. ( https://evds2.tcmb.gov.tr/ )

Bu kapsamda çalışmanın temel amaçları:
- Gayrimenkul sektörünü etkileyen makro değişkenleri anlamak, geleceğe yönelik tahminlemeler yapmak,
- Zaman içindeki eğilimleri ve yapısal dönüşümleri incelemek,
- Veriye dayalı görselleştirmeler ve analizlerle sektörel farkındalık oluşturmaktır.

In [1]:
pip install evds

In [2]:
!pip install plotly

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evds import evdsAPI
import plotly.express as px

In [4]:
evds=evdsAPI('8oRvZm8KVV')

#Ulaşabileceğimiz ana kategoriler aşağıdadır.

In [5]:
evds.main_categories

,CATEGORY_ID,TOPIC_TITLE_TR
0,1,PİYASA VERİLERİ (TCMB)
1,2,KURLAR (TCMB)
2,3,FAİZ VE KÂR PAYI İSTATİSTİKLERİ (TCMB)
3,4,AYLIK PARA VE BANKA İSTATİSTİKLERİ (TCMB)
4,5,MENKUL KIYMET İSTATİSTİKLERİ (TCMB)
5,12,FİNANSAL HİZMETLER ANKETİ (TCMB)
6,13,MERKEZ BANKASI BİLANÇO VERİLERİ (TCMB)
7,14,FİYAT ENDEKSLERİ
8,15,İKTİSADİ YÖNELİM ANKETİ (TCMB)
9,18,ULUSLARARASI YATIRIM POZİSYONU (TCMB)


## 🔍 Bu analizde kullanılabilecek veri grupları

EVDS ana kategorilerinden, Türkiye gayrimenkul / konut piyasasını anlamak için özellikle şu başlıklar kullanılmaya uygundur:

- **Konut Fiyat Endeksi (TCMB) – (CAT_ID: 26)**  
  - Konut fiyatlarındaki aylık ve yıllık değişimi takip etmek için ana veri kaynağıdır.  
  - Reel / nominal konut fiyatı, bölgesel endeksler gibi seriler içerir.

- **Konut ve İnşaat İstatistikleri (TCMB) – (CAT_ID: 28)**  
  - Konut satışları, yapı ruhsatı, yapı kullanma izinleri gibi arz tarafını gösteren seriler.  
  - İnşaat sektöründeki aktiviteyi ve yoğunluğu analiz etmek için kullanılacaktır.

- **Piyasa Verileri, Kurlar, Faiz İstatistikleri (TCMB) – (CAT_ID: 1, 2, 3)**  
  - Politika faizi, kredi faizleri, döviz kurları gibi makro finansal göstergeler.  
  - Konut talebini etkileyen finansman koşullarını (konut kredisi maliyeti vb.) açıklamak için yardımcı değişkenler olarak kullanılacaktır.

- **Banka Kredileri Eğilim Anketi / Finansal Hesaplar – (CAT_ID: 27, 30 vb.)**  
  - Banka kredilerinin kompozisyonu, konut kredisi hacmi ve kredi koşulları hakkında bilgi sağlar.  
  - Konut talebi ile kredi genişlemesi arasındaki ilişkiyi incelemek için değerlendirilebilir.

Bu veri grupları birlikte kullanıldığında,  
- konut fiyatlarının seviyesi ve oynaklığı,  
- konut satışlarının zaman içindeki seyri,  
- faiz, kur ve kredi koşullarının konut talebine etkisi,  
- makroekonomik büyüme ile gayrimenkul sektörü arasındaki ilişki  

gibi başlıklar detaylı şekilde analiz edilebilecektir.

#Dilediğimiz anakategoriye ait CATEGORY_ID'yi kullanarak alt kategorilere ulaşabiliriz.

In [6]:
evds.get_sub_categories(28)

,CATEGORY_ID,DATAGROUP_CODE,DATAGROUP_NAME
0,28,bie_akonutsat1,Konut Satış İstatistikleri - Toplam Satışlar (...
1,28,bie_akonutsat2,Konut Satış İstatistikleri - İpotekli Satışlar...
2,28,bie_akonutsat3,Konut Satış İstatistikleri - İlk El Satış (TÜİK)
3,28,bie_akonutsat4,Konut Satış İstatistikleri - İkinci El Satış (...
4,28,bie_inypkl2,İnşaat İstatistikleri-Yapı Kullanma İzin Belge...
5,28,bie_inyprh2,İnşaat İstatistikleri-Yapı Ruhsatına Göre (TÜİK)


#Seçtiğimiz kategoride bulunan verisetlerini inceleyelim. (örn. Konut Satış İstatistikleri - Toplam Satışlar)

In [7]:
evds.get_series('bie_akonutsat1')

,SERIE_CODE,SERIE_NAME,START_DATE
0,TP.AKONUTSAT1.T4,Konut Satış İstatistikleri _ Ağrı _ Toplam Sat...,01-01-2013
1,TP.AKONUTSAT1.T5,Konut Satış İstatistikleri _ Aksaray _ Toplam ...,01-01-2013
2,TP.AKONUTSAT1.T6,Konut Satış İstatistikleri _ Amasya _ Toplam S...,01-01-2013
3,TP.AKONUTSAT1.T7,Konut Satış İstatistikleri _ Ankara _ Toplam S...,01-01-2013
4,TP.AKONUTSAT1.T8,Konut Satış İstatistikleri _ Antalya _ Toplam ...,01-01-2013
...,...,...,...
77,TP.AKONUTSAT1.T81,Konut Satış İstatistikleri _ Zonguldak _ Topla...,01-01-2013
78,TP.AKONUTSAT1.TOPLAM,Konut Satış İstatistikleri _ Türkiye _ Toplam ...,01-01-2013
79,TP.AKONUTSAT1.K1,Konut Satış İstatistikleri _ Adana _ Toplam Sa...,01-01-2013
80,TP.AKONUTSAT1.T2,Konut Satış İstatistikleri _ Adıyaman _ Toplam...,01-01-2013


#Analizimizde kullanacağımız seriler ile datamızı oluşturalım

In [8]:
KSI_toplam = evds.get_data(['TP.AKONUTSAT1.TOPLAM',"TP.AKONUTSAT1.T40","TP.AKONUTSAT1.T7", "TP.AKONUTSAT1.T41"], startdate="01-01-2015", enddate="26-11-2025")

In [9]:
KSI_toplam.describe()

,TP_AKONUTSAT1_TOPLAM,TP_AKONUTSAT1_T40,TP_AKONUTSAT1_T7,TP_AKONUTSAT1_T41
count,130.000000,130.000000,130.000000,130.000000
mean,117215.323077,20254.676923,11520.076923,6787.146154
std,31589.068836,5666.224731,3346.398616,1961.097870
min,42783.000000,6113.000000,4500.000000,2533.000000
25%,97581.000000,17067.000000,9499.000000,5667.750000
50%,112650.500000,19407.000000,11366.000000,6515.500000
75%,133036.500000,22621.250000,13188.500000,7654.750000
max,229357.000000,40317.000000,26885.000000,15614.000000


In [10]:
KSI_toplam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Tarih                 130 non-null    object 
 1   TP_AKONUTSAT1_TOPLAM  130 non-null    float64
 2   TP_AKONUTSAT1_T40     130 non-null    float64
 3   TP_AKONUTSAT1_T7      130 non-null    float64
 4   TP_AKONUTSAT1_T41     130 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.2+ KB


In [11]:
KSI_toplam

,Tarih,TP_AKONUTSAT1_TOPLAM,TP_AKONUTSAT1_T40,TP_AKONUTSAT1_T7,TP_AKONUTSAT1_T41
0,2015-1,86167.0,16587.0,9570.0,5483.0
1,2015-2,95021.0,17690.0,11063.0,5850.0
2,2015-3,116030.0,21911.0,14105.0,6845.0
3,2015-4,119317.0,23197.0,14001.0,7669.0
4,2015-5,107888.0,21576.0,12816.0,6221.0
...,...,...,...,...,...
125,2025-6,107723.0,17656.0,9428.0,5987.0
126,2025-7,142858.0,23152.0,12491.0,7815.0
127,2025-8,143319.0,21814.0,12419.0,7695.0
128,2025-9,150657.0,24119.0,13417.0,8544.0


In [12]:
#Değişken adlarımızı düzenleyelim
KSI_toplam= KSI_toplam.rename(columns={"TP_AKONUTSAT1_TOPLAM": "Konut Satış Adeti - Türkiye Geneli"})
KSI_toplam = KSI_toplam.rename(columns={"TP_AKONUTSAT1_T40": "Konut Satış Adeti - İstanbul"})
KSI_toplam = KSI_toplam.rename(columns={"TP_AKONUTSAT1_T7": "Konut Satış Adeti- Ankara"})
KSI_toplam = KSI_toplam.rename(columns={"TP_AKONUTSAT1_T41": "Konut Satış Adeti - İzmir"})
KSI_toplam.columns

Index(['Tarih', 'Konut Satış Adeti - Türkiye Geneli',
       'Konut Satış Adeti - İstanbul', 'Konut Satış Adeti- Ankara',
       'Konut Satış Adeti - İzmir'],
      dtype='object')

In [13]:
print(KSI_toplam.isnull().sum())

Tarih                                 0
Konut Satış Adeti - Türkiye Geneli    0
Konut Satış Adeti - İstanbul          0
Konut Satış Adeti- Ankara             0
Konut Satış Adeti - İzmir             0
dtype: int64


In [14]:
KSI_toplam["Tarih"] = pd.to_datetime(KSI_toplam["Tarih"], format="%Y-%m")
KSI_toplam = KSI_toplam.set_index("Tarih")
KSI_toplam

,Konut Satış Adeti - Türkiye Geneli,Konut Satış Adeti - İstanbul,Konut Satış Adeti- Ankara,Konut Satış Adeti - İzmir
Tarih,,,,
2015-01-01,86167.0,16587.0,9570.0,5483.0
2015-02-01,95021.0,17690.0,11063.0,5850.0
2015-03-01,116030.0,21911.0,14105.0,6845.0
2015-04-01,119317.0,23197.0,14001.0,7669.0
2015-05-01,107888.0,21576.0,12816.0,6221.0
...,...,...,...,...
2025-06-01,107723.0,17656.0,9428.0,5987.0
2025-07-01,142858.0,23152.0,12491.0,7815.0
2025-08-01,143319.0,21814.0,12419.0,7695.0


In [15]:
df_plot = KSI_toplam.reset_index()
df_plot.rename(columns={"index": "Tarih"}, inplace=True)

fig = px.line(
    df_plot,
    x="Tarih",
    y=df_plot.columns[1:],
    title="Aylık Konut Satış Adetleri",
    labels={"value": "Satış Adedi", "variable": "Bölge"}
)

fig.update_layout(hovermode="x unified")
fig.show()

##Günlük Döviz Kurlarını Çekelim

In [16]:
evds.get_sub_categories(2)

,CATEGORY_ID,DATAGROUP_CODE,DATAGROUP_NAME
0,2,bie_dkdovytl,Döviz Kurları
1,2,bie_dkefkytl,Efektif Kurlar
2,2,bie_dkkurbil,TCMB Tarafından Alım Satıma Konu Olmayan Döviz...
3,2,bie_rktufey,Kurlar-Reel Efektif Döviz Kuru-TÜFE Bazlı (200...
4,2,bie_rkufey,Kurlar-Reel Efektif Döviz Kuru-Yİ-ÜFE Bazlı (2...
5,2,bie_redkurigm,Kurlar-Reel Efektif Döviz Kuru-Birim İş Gücü M...


In [17]:
evds.get_series('bie_dkdovytl')

,SERIE_CODE,SERIE_NAME,START_DATE
0,TP.DK.USD.A.YTL,(USD) ABD Doları (Döviz Alış),02-01-1950
1,TP.DK.USD.S.YTL,(USD) ABD Doları (Döviz Satış),02-01-1950
2,TP.DK.EUR.A.YTL,(EUR) Euro (Döviz Alış),04-01-1999
3,TP.DK.EUR.S.YTL,(EUR) Euro (Döviz Satış),04-01-1999
4,TP.DK.ATS.A.YTL,(ATS) Avusturya Şilini (Döviz Alış),03-07-1961
...,...,...,...
93,TP.DK.AZN.S.YTL,(AZN) Azerbaycan Yeni Manatı (Döviz Satış),22-12-2021
94,TP.DK.AZN.C.YTL,(AZN) Azerbaycan Yeni Manatı (Çapraz) (USD/AZN),22-12-2021
95,TP.DK.KZT.A.YTL,(KZT) Kazakistan Tengesi (Döviz Alış),03-11-2025
96,TP.DK.KZT.S.YTL,(KZT) Kazakistan Tengesi (Döviz Satış),03-11-2025


In [18]:
USD_EUR = evds.get_data(['TP.DK.USD.A.YTL',"TP.DK.EUR.A.YTL"], startdate="01-01-2015", enddate="26-11-2025")

In [19]:
USD_EUR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3983 entries, 0 to 3982
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tarih            3983 non-null   object 
 1   TP_DK_USD_A_YTL  2739 non-null   float64
 2   TP_DK_EUR_A_YTL  2739 non-null   float64
dtypes: float64(2), object(1)
memory usage: 93.5+ KB


In [20]:
USD_EUR

,Tarih,TP_DK_USD_A_YTL,TP_DK_EUR_A_YTL
0,01-01-2015,NaN,NaN
1,02-01-2015,2.3269,2.8272
2,03-01-2015,NaN,NaN
3,04-01-2015,NaN,NaN
4,05-01-2015,2.3449,2.8258
...,...,...,...
3978,22-11-2025,NaN,NaN
3979,23-11-2025,NaN,NaN
3980,24-11-2025,42.2819,48.7472
3981,25-11-2025,42.3675,48.8473


In [21]:
# Herhangi bir kolonda NaN olan tüm satırları sil
USD_EUR = USD_EUR.dropna()

In [22]:
USD_EUR

,Tarih,TP_DK_USD_A_YTL,TP_DK_EUR_A_YTL
1,02-01-2015,2.3269,2.8272
4,05-01-2015,2.3449,2.8258
5,06-01-2015,2.3411,2.7931
6,07-01-2015,2.3277,2.7736
7,08-01-2015,2.3255,2.7567
...,...,...,...
3976,20-11-2025,42.2756,48.9495
3977,21-11-2025,42.2882,48.7238
3980,24-11-2025,42.2819,48.7472
3981,25-11-2025,42.3675,48.8473


In [23]:
print(USD_EUR.isnull().sum())

Tarih              0
TP_DK_USD_A_YTL    0
TP_DK_EUR_A_YTL    0
dtype: int64


In [24]:
USD_EUR = USD_EUR.rename(columns={
    "TP_DK_USD_A_YTL": "USD_TL_Kuru",
    "TP_DK_EUR_A_YTL": "EUR_TL_Kuru"
})

USD_EUR.head()


,Tarih,USD_TL_Kuru,EUR_TL_Kuru
1,02-01-2015,2.3269,2.8272
4,05-01-2015,2.3449,2.8258
5,06-01-2015,2.3411,2.7931
6,07-01-2015,2.3277,2.7736
7,08-01-2015,2.3255,2.7567


In [25]:
USD_EUR["Tarih"] = pd.to_datetime(USD_EUR["Tarih"], format="%d-%m-%Y")
USD_EUR = USD_EUR.set_index("Tarih")


In [26]:
# @title
# Günlükten aylığa (ay başı frekansı) ortalama kur
USD_EUR_monthly = USD_EUR.resample("MS").mean()   # MS = Month Start

df_fx_m = USD_EUR_monthly.reset_index()
df_fx_m.rename(columns={"index": "Tarih"}, inplace=True)

fig = px.line(
    df_fx_m,
    x="Tarih",
    y=["USD_TL_Kuru", "EUR_TL_Kuru"],
    title="Aylık Ortalama USD ve EUR Döviz Kurları (TL)",
    labels={"value": "Kur", "variable": "Döviz Çifti"}
)

fig.update_layout(hovermode="x unified")
fig.show()

In [27]:
evds.get_sub_categories(26)

,CATEGORY_ID,DATAGROUP_CODE,DATAGROUP_NAME
0,26,bie_kfe,Konut Fiyat Endeksi (KFE) (2023=100)
1,26,bie_ykfe,Yeni Konutlar Fiyat Endeksi (YKFE) (2023=100)
2,26,bie_yokfend,Yeni Olmayan Konutlar Fiyat Endeksi (YOKFE) (2...
3,26,bie_birimfiyat,Konut Birim Fiyatları


In [28]:
evds.get_series('bie_kfe')

,SERIE_CODE,SERIE_NAME,START_DATE
0,TP.KFE.TR,Konut Fiyat Endeksi (KFE),01-01-2010
1,TP.KFE.TR10,TR10 (İstanbul),01-01-2010
2,TP.KFE.TR51,TR51 (Ankara),01-01-2010
3,TP.KFE.TR31,TR31 (İzmir),01-01-2010
4,TP.KFE.TR21,"TR21 (Edirne, Kırklareli, Tekirdağ)",01-01-2010
5,TP.KFE.TR22,"TR22 (Balıkesir, Çanakkale)",01-01-2010
6,TP.KFE.TR32,"TR32 (Aydın, Denizli, Muğla)",01-01-2010
7,TP.KFE.TR33,"TR33 (Afyonkarahisar, Kütahya, Manisa, Uşak)",01-01-2010
8,TP.KFE.TR41,"TR41 (Bursa, Eskişehir, Bilecik)",01-01-2010
9,TP.KFE.TR42,"TR42 (Bolu, Kocaeli, Sakarya, Yalova, Düzce)",01-01-2010


In [29]:
KFE = evds.get_data(['TP.KFE.TR',"TP.KFE.TR10","TP.KFE.TR51", "TP.KFE.TR31"], startdate="01-01-2015", enddate="26-11-2025")

In [30]:
KFE

,Tarih,TP_KFE_TR,TP_KFE_TR10,TP_KFE_TR51,TP_KFE_TR31
0,2015-1,7.65,8.23,7.69,6.57
1,2015-2,7.78,8.40,7.82,6.66
2,2015-3,7.91,8.62,7.93,6.78
3,2015-4,8.02,8.81,8.09,6.83
4,2015-5,8.12,9.08,8.07,6.91
...,...,...,...,...,...
125,2025-6,185.96,173.13,203.64,183.62
126,2025-7,187.78,174.17,206.37,181.93
127,2025-8,192.44,179.45,212.28,186.14
128,2025-9,195.66,182.07,217.30,191.30


In [31]:
KFE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Tarih        130 non-null    object 
 1   TP_KFE_TR    130 non-null    float64
 2   TP_KFE_TR10  130 non-null    float64
 3   TP_KFE_TR51  130 non-null    float64
 4   TP_KFE_TR31  130 non-null    float64
dtypes: float64(4), object(1)
memory usage: 5.2+ KB


In [32]:
# @title
# 1) Kolon isimlerini düzenleyelim
KFE = KFE.rename(columns={
    "TP_KFE_TR":   "KFE_Türkiye",
    "TP_KFE_TR10": "KFE_İstanbul",
    "TP_KFE_TR51": "KFE_Ankara",
    "TP_KFE_TR31": "KFE_İzmir"
})

# 2) Tarih'i datetime yapıp index'e alalım
KFE["Tarih"] = pd.to_datetime(KFE["Tarih"], format="%Y-%m")
KFE = KFE.set_index("Tarih")

# 3) Interaktif grafik için index'i tekrar kolona al
df_kfe = KFE.reset_index()

fig = px.line(
    df_kfe,
    x="Tarih",
    y=["KFE_Türkiye", "KFE_İstanbul", "KFE_Ankara", "KFE_İzmir"],
    title="Konut Fiyat Endeksi (Türkiye ve 3 Büyük İl)",
    labels={"value": "KFE (2010=100)", "variable": "Bölge"}
)

fig.update_layout(hovermode="x unified")
fig.show()

**Konut Fiyat Endeksi Yıllık Değişim Oranı (YoY %)**

- 2021–2023 döneminde tüm bölgelerde **tarihi bir fiyat artış hızlanması** yaşanmıştır.
- Yıllık artış oranı bazı şehirlerde **%200’lere** kadar çıkmış, bu da konut fiyatlarının bir önceki yılın aynı ayına göre yaklaşık **3 katına** ulaştığını göstermektedir.
- En sert yükseliş **2022 yılı** içinde görülmüştür. Bu dönem:
  - Yüksek enflasyon,
  - Dövizde artış
  gibi makro faktörlerin etkisiyle fiyatların hızla yükseldiği bir dönemdir.
- 2023 sonrasında artış hızı belirgin şekilde **yavaşlamış**, grafik daha yatay bir yapıya dönmüştür.
- İstanbul, Ankara ve İzmir arasındaki fark YoY yüzdelerinde oldukça yakın seyretmiştir; yani **üç büyük il aynı ekonomik dalgadan benzer şekilde etkilenmiştir**.

In [33]:
# @title
KFE_yoy = KFE.pct_change(12) * 100   # 12 ay gecikmeli yıllık değişim

df_yoy = KFE_yoy.reset_index()

fig = px.line(
    df_yoy,
    x="Tarih",
    y=["KFE_Türkiye", "KFE_İstanbul", "KFE_Ankara", "KFE_İzmir"],
    title="Konut Fiyat Endeksi Yıllık Değişim Oranı (%)",
    labels={"value": "Yıllık Değişim (%)", "variable": "Bölge"}
)
fig.update_layout(hovermode="x unified")
fig.show()


 **Şehirlerin Türkiye Ortalama Endeksine Göre Fiyat Farkı**

Bu grafik, üç büyük ilin konut fiyatlarının **Türkiye genelinden ne kadar ayrıştığını** göstermektedir.

- 2015–2021 döneminde şehirler arasında büyük bir fark yoktur; fiyatlar **Türkiye ortalamasına çok yakın** seyretmiştir.
- 2022 sonrasında şehirler arasında anlamlı ayrışmalar başlar:
  - **İstanbul** bir süre Türkiye ortalamasının üzerinde kalsa da 2023'ten itibaren ortalamanın **altına düşmeye** başlamıştır.
  - **Ankara** uzun süre ortalamaya yakın seyretmiş, son dönemde ortalamaya göre hafif yukarı yönlü bir ayrışma göstermiştir.
  - **İzmir** özellikle 2023 sonrası ortalamaya göre **belirgin şekilde daha pahalı hale gelmiştir**. Son değerlerde İzmir’in Türkiye ortalamasının yaklaşık **20+ endeks puanı üzerinde** olduğu görülmektedir.
- Bu ayrışma, şehir bazlı talep farklılıkları, göç hareketleri, yatırım davranışları ve bölgesel arz koşullarının etkisini yansıtmaktadır.

In [34]:
# @title
KFE_diff = KFE.copy()
for col in ["KFE_İstanbul", "KFE_Ankara", "KFE_İzmir"]:
    KFE_diff[col] = KFE[col] - KFE["KFE_Türkiye"]

df_diff = KFE_diff.reset_index()

fig = px.line(
    df_diff,
    x="Tarih",
    y=["KFE_İstanbul", "KFE_Ankara", "KFE_İzmir"],
    title="Şehirlerin Türkiye Ortalamasına Göre Fiyat Farkı (KFE Puanı)",
    labels={"value": "Fark (Endeks Puanı)", "variable": "Bölge"}
)
fig.update_layout(hovermode="x unified")
fig.show()


In [35]:
evds.get_series('bie_birimfiyat')

,SERIE_CODE,SERIE_NAME,START_DATE
0,TP.BIRIMFIYAT.TR,Türkiye Konut Birim Fiyatları_TL/m2,01-01-2010
1,TP.BIRIMFIYAT.IST,İstanbul Konut Birim Fiyatları_TL/m2,01-01-2010
2,TP.BIRIMFIYAT.ANK,Ankara Konut Birim Fiyatları_TL/m2,01-01-2010
3,TP.BIRIMFIYAT.IZM,İzmir Konut Birim Fiyatları_TL/m2,01-01-2010
4,TP.BIRIMFIYAT.ADANA,Adana Konut Birim Fiyatları_TL/m2,01-01-2010
...,...,...,...
72,TP.BIRIMFIYAT.YALOVA,Yalova Konut Birim Fiyatları_TL/m2,01-01-2010
73,TP.BIRIMFIYAT.KARABUK,Karabük Konut Birim Fiyatları_TL/m2,01-01-2010
74,TP.BIRIMFIYAT.KILIS,Kilis Konut Birim Fiyatları_TL/m2,01-01-2014
75,TP.BIRIMFIYAT.OSMANIYE,Osmaniye Konut Birim Fiyatları_TL/m2,01-01-2010


In [36]:
KBF = evds.get_data(['TP.BIRIMFIYAT.TR',"TP.BIRIMFIYAT.IST","TP.BIRIMFIYAT.ANK", "TP.BIRIMFIYAT.IZM"], startdate="01-01-2015", enddate="26-11-2025")

In [37]:
KBF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tarih              43 non-null     object 
 1   TP_BIRIMFIYAT_TR   43 non-null     float64
 2   TP_BIRIMFIYAT_IST  43 non-null     float64
 3   TP_BIRIMFIYAT_ANK  43 non-null     float64
 4   TP_BIRIMFIYAT_IZM  43 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.8+ KB


In [38]:
KBF

,Tarih,TP_BIRIMFIYAT_TR,TP_BIRIMFIYAT_IST,TP_BIRIMFIYAT_ANK,TP_BIRIMFIYAT_IZM
0,2015-Q1,1924.0,3634.5,1496.6,1777.8
1,2015-Q2,2030.3,3870.8,1577.1,1857.6
2,2015-Q3,2105.1,3995.5,1634.3,1982.4
3,2015-Q4,2217.2,4343.3,1680.5,1995.1
4,2016-Q1,2303.1,4347.2,1698.6,2060.2
5,2016-Q2,2408.7,4581.7,1736.7,2184.6
6,2016-Q3,2527.1,4872.3,1750.6,2257.2
7,2016-Q4,2561.7,4887.7,1840.6,2340.4
8,2017-Q1,2638.4,4956.2,1852.1,2478.1
9,2017-Q2,2753.8,5212.8,1897.5,2558.5


**Konut m² Fiyatları (TL/m²) – Türkiye ve 3 Büyük İl**

Bu grafik, 2015–2025 döneminde Türkiye geneli, İstanbul, Ankara ve İzmir için konut m² satış fiyatlarının nasıl değiştiğini göstermektedir.

---

### 🔹 1) Genel Trend

- 2015–2020 arasında m² fiyatları görece **ılımlı ve kademeli** bir artış eğilimindedir.
- 2021 sonrası dönemde özellikle 2022–2024 arasında **çok sert bir fiyat sıçraması** görülmektedir.
- Grafik, konut fiyatlarının yalnızca arttığını değil, **artış hızının da zamanla hızlandığını** göstermektedir.
- Son çeyreklerde (2024–2025) seviyeler:
  - Türkiye geneli için yaklaşık **40.000–45.000 TL/m²** bandına,
  - İstanbul için **60.000–70.000 TL/m²** bandına kadar yükselmiştir.

---

### 🔹 2) Şehirler Arası Karşılaştırma

- Tüm dönem boyunca **İstanbul m² fiyatı en yüksek** olan ildir.
- **İzmir**, Türkiye ortalamasının üzerinde seyretmekte ve zamanla İstanbul’a nispeten daha fazla yaklaşan bir profil çizmektedir.
- **Ankara** m² fiyatları, Türkiye ortalamasının bir miktar altında kalmakta; buna rağmen 2022 sonrası dönemde Ankara’da da ciddi bir artış vardır.
- 2021 sonrasında şehirler arası fark **mutlak değer olarak açılmıştır**:
  - İstanbul ile Türkiye ortalaması arasındaki fark binlerce TL/m² seviyesine çıkmıştır.
  - İzmir’in Türkiye ortalamasına göre primi belirginleşmiş,
  - Ankara ise görece daha düşük maliyetli bir büyükşehir olarak konumlanmıştır.

---

### 🔹 3) Dönüm Noktaları

- **2021 Q2–Q3**’ten itibaren tüm serilerde eğim belirgin şekilde yukarı döner:
  - Bu dönem, yüksek enflasyon, kredi koşulları ve döviz hareketleriyle birlikte
    konutun **enflasyondan korunma aracı** olarak talep gördüğü bir dönemdir.
- **2022–2023** döneminde artış en dik seviyesindedir; grafik neredeyse
  **ekponansiyel** bir eğim göstermektedir.
- 2024 sonrası dönemde artış devam etmekle birlikte, bazı çeyreklerde
  artış hızının görece **yavaşladığı** gözlenebilir.

---

### 🔹 4) Ön Çıkarımlar

- Konut m² satış fiyatları hem Türkiye genelinde hem de büyük şehirlerde
  **çok yüksek bir maliyet seviyesine** ulaşmıştır.
- İstanbul ve İzmir, m² bazında yatırım ve oturum için
  **en pahalı iki şehir** olarak öne çıkmaktadır.
- Ankara, fiyat seviyesi açısından daha düşük kalsa da,
  2021 sonrası dönemde buradaki artış da
  **Türkiye ortalamasıyla paralel veya zaman zaman üzerinde** gerçekleşmiştir.


In [39]:
# @title
KBF = KBF.rename(columns={
    "TP_BIRIMFIYAT_TR":  "m2_Türkiye",
    "TP_BIRIMFIYAT_IST": "m2_İstanbul",
    "TP_BIRIMFIYAT_ANK": "m2_Ankara",
    "TP_BIRIMFIYAT_IZM": "m2_İzmir"
})

# Tarih: 2015-Q1 → çeyrek başlangıç tarihi
KBF["Tarih"] = pd.PeriodIndex(KBF["Tarih"], freq="Q").to_timestamp(how="start")

fig = px.line(
    KBF,
    x="Tarih",
    y=["m2_Türkiye", "m2_İstanbul", "m2_Ankara", "m2_İzmir"],
    title="Konut m² Fiyatları (TL/m²) - Türkiye ve 3 Büyük İl (Çeyreklik)",
    labels={"value": "m² Fiyatı (TL/m²)", "variable": "Bölge"}
)

fig.update_layout(hovermode="x unified")
fig.show()

## Kur Tahmini